<a href="https://colab.research.google.com/github/devishi/CF_Project/blob/master/PUCPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Preprocessing

In [0]:
import numpy as np
import pandas as pd
import time
from pandas import DataFrame
from operator import itemgetter
from sklearn.preprocessing import Imputer, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.ensemble import AdaBoostClassifier as AB
from sklearn.cross_validation import cross_val_score as CV
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix
from sklearn import metrics, preprocessing, dummy
print("Yo")

Yo


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
%cd drive/My Drive/Colab Notebooks/
%cd Assignments_sem7/CF_Project

[Errno 2] No such file or directory: 'drive/My Drive/Colab Notebooks/'
/content/drive/My Drive/Colab Notebooks/Assignments_sem7/CF_Project
[Errno 2] No such file or directory: 'Assignments_sem7/CF_Project'
/content/drive/My Drive/Colab Notebooks/Assignments_sem7/CF_Project


## Data Preprocessing

Source of the Dataset : https://github.com/fiatveritas/eHarmony_Project

In [0]:
training_data = pd.read_csv("EH-training-data.csv", names = range(0, 59))
training_labels = pd.read_csv("EH-training-labels.csv", names = range(0, 3))
testing_data = pd.read_csv("EH-testing-data.csv", names = range(0, 59))
testing_labels = pd.read_csv("EH-testing-labels.csv", names = range(0, 3))
print "This set has {} training_data points and {} features.".format(*training_data.shape)
print "This set has {} training_label points and {} features.".format(*training_labels.shape)
print "This set has {} testing_data points and {} features.".format(*testing_data.shape)
print "This set has {} testing_label points and {} features.".format(*testing_labels.shape)


This set has 274654 training_data points and 59 features.
This set has 506688 training_label points and 3 features.
This set has 211810 testing_data points and 59 features.
This set has 439161 testing_label points and 3 features.


In [0]:
# features of the user_data
training_data = training_data.truncate(after = 1000)
training_data = training_data.drop((0),axis=1)
print(training_data.shape)
training_data.head()

(1001, 58)


,1,2,3,4,5,6,7,8,9,10,...,49,50,51,52,53,54,55,56,57,58
0,2,3,1,1620,26,4,1,2,4,5,...,0,0,0,5,5,7,5,6,-117.9970,33.8005
1,2,3,200,1720,38,3,2,1,0,2,...,0,0,1,2,6,6,6,5,-73.8584,40.8424
2,2,3,120,1750,26,3,3,1,0,3,...,0,0,0,4,5,7,5,5,-98.4992,29.6647
3,1,3,200,1850,39,0,1,2,0,2,...,1,0,0,1,5,6,4,3,-72.4706,42.1729
4,1,3,200,1880,44,1,2,2,0,2,...,0,0,0,3,7,6,4,5,-76.3964,42.9142


In [0]:
training_labels.head()

,0,1,2
0,1,7,11883
1,1,7,257487
2,0,7,239226
3,0,7,82094
4,0,7,226461


In [0]:
# match(1) or not(0) ,  user_id1, userid2
##########################################
# user_item_matrix
# CROP LABEL DATASET for user_ID < 10000

crop = 1001

print(training_labels.shape,testing_labels.shape)

train_lab = pd.DataFrame(columns=["match","user1","user2"])
test_lab = pd.DataFrame(columns=["match","user1","user2"])

for i in range(len(training_labels)):
  if(training_labels.iloc[i,1] < crop and training_labels.iloc[i,2] < crop):
    train_lab = train_lab.append({"match" : training_labels.iloc[i,0],"user1":training_labels.iloc[i,1],"user2":training_labels.iloc[i,2]},ignore_index=True)
  
for i in range(len(testing_labels)):
  if(testing_labels.iloc[i,1] < crop and testing_labels.iloc[i,2] < crop):
    test_lab = test_lab.append({"match" : testing_labels.iloc[i,0],"user1":testing_labels.iloc[i,1],"user2":testing_labels.iloc[i,2]},ignore_index=True)

print(train_lab.shape, test_lab.shape)
    



((506688, 3), (439161, 3))
((289, 3), (552, 3))


In [0]:
crop = 1000
train_labn = [ [0] * crop for _ in range(crop)]

for i in range(train_lab.shape[0]):
  print(i,train_lab.iloc[i,1],train_lab.iloc[i,2],train_lab.iloc[i,0])
  train_labn[train_lab.iloc[i,1]][train_lab.iloc[i,2]] = train_lab.iloc[i,0]
  train_labn[train_lab.iloc[i,2]][train_lab.iloc[i,1]] = train_lab.iloc[i,0]
print(train_labn)  

In [0]:
for i in range(285,train_lab.shape[0]):
  print(i,train_lab.iloc[i,1],train_lab.iloc[i,2],train_lab.iloc[i,0])
  train_labn[train_lab.iloc[i,1]][train_lab.iloc[i,2]] = train_lab.iloc[i,0]
  train_labn[train_lab.iloc[i,2]][train_lab.iloc[i,1]] = train_lab.iloc[i,0]
print(train_labn)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [0]:
for i in range(len(train_labn)):
  if(max(train_labn[i])>0):
    print(i)

In [0]:
import pickle
item_user = open('item_user.pkl', 'wb')
pickle.dump(train_labn,item_user)
item_user.close()
# mydict2 = pickle.load(pkl_file)

 ## train_labn-> ITEM*USER MATRIX
 ## train_data -> item*item matrix
 

In [0]:
print(training_data[1],len(train_labn[0]))

In [0]:
# user  *  item 
from sklearn.metrics.pairwise import cosine_similarity
print(training_data.shape)
# user1 *  user2
training_data = training_data.values

print(training_data[1].reshape(1,-1))
cosa = cosine_similarity(training_data[1].reshape(1, -1), training_data[3].reshape(1, -1))
print(training_data[1].reshape(1, -1), cosa)

(1001, 58)
[[ 2.00000e+00  3.00000e+00  2.00000e+02  1.72000e+03  3.80000e+01
   3.00000e+00  2.00000e+00  1.00000e+00  0.00000e+00  2.00000e+00
   5.00000e+00  1.00000e+00  0.00000e+00  0.00000e+00  3.99000e+02
   5.10000e+01  3.50000e+01  0.00000e+00  4.00000e+00  4.00000e+00
   3.00000e+00  6.00000e+00  1.00000e+00  2.00000e+00  4.10000e+01
   0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00
   0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00
   0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00
   0.00000e+00  1.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00
   0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00
   1.00000e+00  2.00000e+00  6.00000e+00  6.00000e+00  6.00000e+00
   5.00000e+00 -7.38584e+01  4.08424e+01]]
(array([[ 2.00000e+00,  3.00000e+00,  2.00000e+02,  1.72000e+03,
         3.80000e+01,  3.00000e+00,  2.00000e+00,  1.00000e+00,
         0.00000e+00,  2.00000e+00,  5.00000e+00,  1.00000e+00,
         0.00000

In [0]:
# user * user
import pickle
from sklearn.metrics.pairwise import cosine_similarity
item_er = open('user_user_sim.pkl', 'wb')
train_data = [ [0] * crop for _ in range(crop)]
# training_data = training_data.values
for i in range(crop):
  if(i%500 == 0):
    pickle.dump(train_data,item_er)
  for j in range(crop):
    if(train_data[i][j] == 0):
      cos = cosine_similarity(training_data[i].reshape(1, -1), training_data[j].reshape(1, -1))
#       print(cos)
      train_data[i][j] = cos
      train_data[j][i] = cos
# print(train_data.shape)

In [0]:
for i in range(50):
  print(train_data[i])

In [0]:
import pickle
item_er = open('user_user_sim.pkl', 'wb')
pickle.dump(train_data,item_er)
item_er.close()

In [0]:
with open('item_item.txt', 'w') as output:
    output.write(str(train_data))

In [0]:
with open('user_item.txt', 'w') as output:
    output.write(str(train_labn))

In [0]:
# item * item
  
test_data = pd.DataFrame(columns=["match","user1","user2"])

for i in test_lab.columns:
  for j in test_lab.columns:
    cos = cosine_similarity(training_data.iloc[i].reshape(1,-1),training_data.iloc[j].reshape(1,-1))
    test_data = test_data.append({"match": cos,"user1":i,"user2":j},ignore_index=True)

print(test_data.shape)

## Algorithm - PUCPI

In [0]:
!pip install surprise

In [0]:
import time
import scipy.io as sio
import numpy as np
import random
import pandas as pd
import math
from math import sqrt
from pandas import DataFrame
from numpy import mean

from sklearn import metrics, preprocessing, dummy
from sklearn.model_selection import train_test_split, KFold
from sklearn.svm import SVC
from sklearn.preprocessing import Imputer, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier as RF, AdaBoostClassifier as AB
from sklearn.cross_validation import cross_val_score as CV, StratifiedShuffleSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, confusion_matrix


from operator import itemgetter
from surprise import Reader
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor

In [0]:
train1 = ['rating', 'user1', 'user2']
train_label = pd.read_csv('EH-training-labels.csv', sep=',', names=train1, encoding='latin-1', header = None)
test1 = ['rating', 'user1', 'user2']
test_label = pd.read_csv('EH-testing-labels.csv', sep=',', names=test1, encoding='latin-1', header = None)

In [0]:
train_features = pd.read_csv('EH-training-data.csv', sep=',', encoding='latin-1', header = None)
test_features = pd.read_csv('EH-testing-data.csv', sep=',', encoding='latin-1', header = None)

In [0]:
df = pd.DataFrame(train_label)

# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(0, 1))
train1 = ['user1', 'user2', 'rating']
df = df[train1]
# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[['user1', 'user2', 'rating']], reader)

In [0]:
s0 = [train_label.iloc[0,0]]
s0.extend(train_features.iloc[train_label.iloc[0,1] - 1, 1:].tolist())
s0.extend(train_features.iloc[train_label.iloc[0,2] - 1, 1:].tolist())
data = [s0]
train = pd.DataFrame(data)
ltrain = [0]
for j in range(1, 30000):
  t = 0
  while(t!=1):
    d = random.randint(1, len(train_label))
    if d not in ltrain:
      t = 1
      i = d
  s0 = [train_label.iloc[i,0]]
  s0.extend(train_features.iloc[train_label.iloc[i,1] - 1, 1:].tolist())
  s0.extend(train_features.iloc[train_label.iloc[i,2] - 1, 1:].tolist())
  data = [s0]
  train = train.append(data)

In [0]:
s0 = [test_label.iloc[0,0]]
s0.extend(test_features.iloc[test_label.iloc[0,1] - 1, 1:].tolist())
s0.extend(test_features.iloc[test_label.iloc[0,2] - 1, 1:].tolist())
data = [s0]
test = pd.DataFrame(data)
ltrain = [0]
for j in range(1, 30000):
  t = 0
  while(t!=1):
    d = random.randint(1, len(test_label))
    if d not in ltrain:
      t = 1
      i = d
  s0 = [test_label.iloc[i,0]]
  s0.extend(test_features.iloc[test_label.iloc[i,1] - 1, 1:].tolist())
  s0.extend(test_features.iloc[test_label.iloc[i,2] - 1, 1:].tolist())
  data = [s0]
  test = test.append(data)

In [0]:
train_y = train.iloc[:,0]
train = train.iloc[:,1:]
test = test.dropna()
test_y = test.iloc[:,0]
test = test.iloc[:,1:]

from sklearn.svm import SVC
clf = SVC(gamma='auto')
clf.fit(train, train_y) 

pred = clf.predict(test)

In [0]:
pred.shape 

(30000,)

In [0]:
test_y.shape

(30000,)

### AUC: 0.5

In [0]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import auc
from sklearn import metrics
# accuracy_score(test_y,pred)
fpr, tpr, thresholds = metrics.roc_curve(test_y, pred)
auc_score = auc(fpr,tpr)
print(auc_score)

0.5
